## Keras-ImageDataGenerator

## 『本次練習內容』
#### 學習使用Keras-ImageDataGenerator 與 Imgae Augmentation 做圖像增強

## 『本次練習目的』
  #### 熟悉Image Augmentation的實作方法
  #### 瞭解如何導入Imgae Augmentation到原本NN架構中

## 第一 Part

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

# define Augmentation, img_gen is a generator
img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, rotation_range=10, 
                             width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=False, dtype=np.float32)
width = 224
height = 224
batch_size = 4

img = cv2.imread('img/Tano.JPG')  
img = cv2.resize(img, (width, height)) #  resize img to (224, 224)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 original is BGR，convert to RGB
img_origin = img.copy() # img_origin is RGB (224,224,3)
img = np.array(img, dtype=np.float32) # img is (224,224,3)
img_combine = np.array([img,img,img,img], dtype=np.uint8) # let (224,224,3) become to (4,224,224,3)
batch_gen = img_gen.flow(img_combine, batch_size=4)       # batch_gen is a iterator

assert next(batch_gen).shape==(batch_size, height, width, 3)

plt.figure(figsize=(20,10))
for batch,i in zip(batch_gen, [2,3,4,5]) :
    plt.subplot(1, 5, 1)
    plt.imshow(img_origin) # 第1張: original image (RGB)
    
    plt.subplot(1, 5, i)   # 第2~5張
    plt.imshow(batch[0, :, :, :].astype(np.uint8))  # batch_0
    plt.imshow(batch[1, :, :, :].astype(np.uint8))  # batch_1
    plt.imshow(batch[2, :, :, :].astype(np.uint8))  # batch_2
    plt.imshow(batch[3, :, :, :].astype(np.uint8))  # batch_3
    plt.axis('off')

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


## 示範如何導入ImageDataGenerator到Keras訓練中

In [3]:
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10          # for Downloading cifar10 data set 
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

input_shape = (64, 64, 3)

# 建立 CNN 層模型 (Build CNN model)
classifier = Sequential()

# 1st convolution layer(input layer) :
# Convolution2D(numbers of convolution kernel, kernel_row, kernel_column)
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=input_shape)) 
# output_shape=(None, 32, 32, 32)
classifier.add(MaxPooling2D(pool_size=(2,2))) 

# BatchNormalization : normalize the input

classifier.add(BatchNormalization()) #output_shape=(None, 16, 16, 32)

# 2nd convolution layer(hidden layer) 
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')) # output_shape=(None, 16, 16, 32)
classifier.add(MaxPooling2D(pool_size=(2,2)))  
# 2nd convolution layer:Total params(參數量)=(Kernel_row*column*channels+1)*Kernel numbers(kernel張數)=(3*3*32+1)*32=9248

# BatchNormalization : normalize the input
classifier.add(BatchNormalization())  #output_shape=(None, 8, 8, 32)

classifier.add(Flatten())  #output_shape=(None, 2048),

# FC layer (1st layer)
classifier.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.l2(0.001))) 
# output_shape=(None, 100)

# BatchNormalization : normalize the input
classifier.add(BatchNormalization()) # output_shape=(None, 100) 

# dropout rate : 50 %
classifier.add(Dropout(rate=0.5)) # output_shape=(None, 100)

# FC layer (2nd layer)
classifier.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.l2(0.001))) 
# output_shape=(None, 100)

# BatchNormalization : normalize the input
classifier.add(BatchNormalization()) # output_shape=(None, 100) 

# dropout rate : 30 %
classifier.add(Dropout(rate=0.3))  # output_shape=(None, 100)

# FC layer (output layer) 
classifier.add(Dense(units=2, activation='softmax')) 
# output_shape=(None, 2)

## =============================================================

#Training Generator
train_data_gen = ImageDataGenerator(rescale = 2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
#Test Generator，只需要Rescale，不需要其他增強
test_data_gen = ImageDataGenerator(rescale = 1./255)

training_set = train_data_gen.flow_from_directory('img/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

test_set = test_data_gen.flow_from_directory('img/test_set',
                                            target_size = (64, 64),
                                            batch_size = 8,
                                            class_mode = 'categorical')

# training model 
batch_size_1 = 4

# compile( 優化器:optimizer='adam', 損失函數loss function:categorical_crossentropy, 模型評估指標:metrics=['accuracy'] )
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# EarlyStopping() 
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) # earlystop

classifier.fit_generator(training_set, steps_per_epoch = len(next(training_set)[0]) / batch_size_1, epochs = 20,
                         validation_data = test_set, validation_steps = len(next(test_set)[0]) / batch_size_1,
                         callbacks = [earlystop])

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'img/training_set'

In [ ]:
# 預測新照片
from keras.preprocessing import image as image_utils
# PIL image
test_image = image_utils.load_img('img/new_images/new_picture.jpg', target_size=(64, 64)) 
print(test_image)
test_image = image_utils.img_to_array(test_image) # image to array (3D), (64,64,3)
print(test_image.shape)
test_image = np.expand_dims(test_image, axis=0)   # 3D expand to 4D (1,64,64,3)
print(test_image.shape)


result = classifier.predict_on_batch(test_image)
result  # input class is dog, and result get the probability  is dog

## 練習使用Image Augmentation
- 使用單項增強

In [ ]:
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import numpy as np
import cv2

img = cv2.imread('Tano.JPG')  
img = cv2.resize(img, (224,224))##改變圖片尺寸
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Cv2讀進來是BGR，轉成RGB
img_origin=img.copy()
img= np.array(img, dtype=np.float32)

images = np.random.randint(0, 255, (5, 224, 224, 3), dtype=np.uint8)##創造一個array size==(5, 224, 224, 3)

flipper = iaa.Fliplr(1.0) #水平翻轉機率==1.0
images[0] = flipper.augment_image(img) 

vflipper = iaa.Flipud('''填入''') #垂直翻轉機率40%
images[1] = vflipper.augment_image(img) 

blurer = iaa.GaussianBlur(3.0)
images[2] = blurer.augment_image(img) # 高斯模糊圖像( sigma of 3.0)

translater = iaa.Affine(translate_px={"x": -16}) #向左橫移16個像素
images[3] = translater.augment_image(img) 

scaler = iaa.Affine(scale={"y":'''填入'''}) # 縮放照片，區間(0.8-1.2倍)
images[4] = scaler.augment_image(img)

i=1
plt.figure(figsize=(20,20))
for image in images:
    plt.subplot(1, 6, 1)
    plt.imshow(img_origin.astype(np.uint8))
    plt.subplot(1, 6, i+1)
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')
    i+=1

## 第二Part

### 打包多種Augmentation
#### 請自行練習新增以及改變Augmentation內容
#### 可參考Github: https://github.com/aleju/imgaug

## 包裝自定義Augmentation 與 Imgaug Augmentation

## 鎖住隨機性-主要用在Semantic segmentation中